# Regra de negócios aplicada ao Olist Dataset

A regra de negócios neste notebook é uma simulação para extrair insights dos dados da camada Silver e adiciona-los à camada Gold deste projeto. Este notebook permitirá construir dashboards e relatórios posteriormente.

## Business Rules Checklist

Abaixo temos um checklist de regras de negócio simulado, sugestões de o que será extraído ao decorrer deste notebook, sabendo disso, segue a lista do checklist da regra de negócios:

Com base na estrutura dos datasets:


📌 2. **Vendedores (sellers)**

    ✅ Regra de Negócio: Vendedores com alta taxa de cancelamento ou avaliações negativas devem passar por auditoria e treinamentos.

📌 3. **Clientes (customers)**

    ✅ Regra de Negócio: Oferecer promoções direcionadas para clientes de regiões com alto ticket médio para aumentar recorrência de compras.

📌 4. **Geolocalização (geolocation)**

    ✅ Regra de Negócio: Expandir vendedores para regiões com alta demanda e poucos fornecedores para equilibrar a oferta de produtos.

📌 5. **Itens dos Pedidos (order_items)**

    ✅ Regra de Negócio: Produtos com baixa margem de lucro e alto custo de frete devem ser reavaliados no catálogo.

📌 6. **Pagamentos (order_payments)**

    ✅ Regra de Negócio: Estimular métodos de pagamento que geram maior ticket médio e menor taxa de cancelamento.

📌 7. **Avaliações (order_reviews)**

    ✅ Regra de Negócio: Vendedores com avaliações abaixo de 3 estrelas devem ser notificados para ações corretivas.

📌 8. **Pedidos (orders)**

    ✅ Regra de Negócio: Monitorar pedidos com atraso recorrente e penalizar transportadoras que descumprirem prazos.

📌 9. **Produtos (products)**

    ✅ Regra de Negócio: Produtos com maior volume de vendas devem ter prioridade no estoque para evitar rupturas.

📌 10. **Leads Qualificados (leads_qualified)**

    ✅ Regra de Negócio: Leads qualificados com origem em campanhas de alto desempenho devem receber prioridade no atendimento.

📌 11. **Leads Fechados (leads_closed)**

    ✅ Regra de Negócio: Focar esforços de vendas nos segmentos de clientes com maior taxa de conversão e ticket médio.

In [3]:
import pandas as pd
import duckdb

def pandas_sql(query: str) -> pd.DataFrame:
    """Executa uma consulta SQL e retorna um DataFrame."""
    conn = duckdb.connect()
    result = conn.sql(query).fetchdf()
    conn.close()
    return result

###################################################################

pandas_sql("SELECT * FROM delta_scan('../delta_lake/silver/geolocation') LIMIT 10")
# query_sql_pandas("SELECT * FROM delta_scan('../delta_lake/silver/order_reviews') LIMIT 10")

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,99940,-28.057239,-51.859525,ibiaçá,RS
1,99950,-28.066815,-52.012488,tapejara,RS
2,99950,-28.070611,-52.013596,tapejara,RS
3,99950,-28.065525,-52.006841,tapejara,RS
4,99940,-28.059619,-51.855084,ibiaca,RS
5,01012,-23.548946,-46.634671,sao paulo,SP
6,01039,-23.541883,-46.639919,sao paulo,SP
7,01009,-23.546935,-46.636588,sao paulo,SP
8,01046,-23.545884,-46.643163,sao paulo,SP
9,01050,-23.548756,-46.642789,sao paulo,SP


In [14]:
pandas_sql("SELECT * FROM delta_scan('../delta_lake/bronze/customers') LIMIT 10")
# pandas_sql("SELECT * FROM delta_scan('../delta_lake/silver/customers_silver') LIMIT 10")

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,09790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,01151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,08775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP
5,879864dab9bc3047522c92c82e1212b8,4c93744516667ad3b8f1fb645a3116a4,89254,jaragua do sul,SC
6,fd826e7cf63160e536e0908c76c3f441,addec96d2e059c80c30fe6871d30d177,04534,sao paulo,SP
7,5e274e7a0c3809e14aba7ad5aae0d407,57b2a98a409812fe9618067b6b8ebe4f,35182,timoteo,MG
8,5adf08e34b2e993982a47070956c5c65,1175e95fb47ddff9de6b2b06188f7e0d,81560,curitiba,PR
9,4b7139f34592b3a31687243a302fa75b,9afe194fb833f79e300e37e580171f22,30575,belo horizonte,MG


In [ ]:
import duckdb
from typing import Optional
import pandas as pd

def sql_db(query: str, db_path: Optional[str] = ':memory:') -> pd.DataFrame:
    conn = duckdb.connect(db_path)
    result = conn.sql(query).fetchdf()
    conn.close()
    return result  # return dataframe

df = sql_db("SELECT * FROM customers LIMIT 10", db_path="../data/olist.sqlite")
df


,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP
5,879864dab9bc3047522c92c82e1212b8,4c93744516667ad3b8f1fb645a3116a4,89254,jaragua do sul,SC
6,fd826e7cf63160e536e0908c76c3f441,addec96d2e059c80c30fe6871d30d177,4534,sao paulo,SP
7,5e274e7a0c3809e14aba7ad5aae0d407,57b2a98a409812fe9618067b6b8ebe4f,35182,timoteo,MG
8,5adf08e34b2e993982a47070956c5c65,1175e95fb47ddff9de6b2b06188f7e0d,81560,curitiba,PR
9,4b7139f34592b3a31687243a302fa75b,9afe194fb833f79e300e37e580171f22,30575,belo horizonte,MG
